In [1]:
import pandas as pd
import csv

def csv_to_sentences_inference(input_file, output_file):
    with open(input_file, 'r') as csvfile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(csvfile)
        writer = csv.writer(outfile)

        for row in reader:
            # sentence = ", ".join(f"Column {index}: {value}" for index, value in enumerate(row.values())) + " "
            sentence = ", ".join(f"Column {index}: {value}" for index, value in enumerate(row.values())) + " "
            writer.writerow([sentence])


def read_csv_into_list_inference(filename):
    data = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Skip the header row
        for row in reader:
            # print('row', row)
            data.append(row[0])
            # data.append(row)
    return data


from transformers import pipeline
# fill = pipeline('fill-mask', model='the_TREB_model', tokenizer='the_tokenizer', device=0)
fill = pipeline('fill-mask', model='the_TREB_model_the_ultimate', tokenizer='the_tokenizer_the', device=0)
fill

import re


0

In [ ]:
filename = 'cal_dataframe_normalized_emptied_0.csv'
filename_result = filename.replace('.csv', '_result.csv')
csv_to_sentences_inference(filename, filename_result)
empty_column_indices = [0]  
timu = read_csv_into_list_inference(filename_result)
timu[0]

In [ ]:

text_data = []

for tim in timu:
    res = fill(tim) 
    print('res', res)
    # print('res', res[0])
    # print('res', res[0]['token_str'])
    mask_position = [m.start() for m in re.finditer('[MASK]', tim)]
    # print('mask_position', mask_position)

    # Replace each 'MASK' with the corresponding string
    modified_text = ""
    modified_text += tim[0:mask_position[1]] + res[0]['token_str']
    modified_text += tim[mask_position[-1]+1:]
    text_data.append(modified_text.replace('\n', ''))

with open(filename.replace('.csv', '') + '_imputed.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))



In [ ]:
# Function to read a row from the txt file and parse it into a dictionary
def parse_row(row):
    data = {}
    for column_value in row.split(', '):
        key, value = column_value.split(': ')
        data[key] = float(value)  # Convert values to float
    return data

# Read the txt file and create a list of dictionaries
with open(filename.replace('.csv', '') + '_imputed.txt', 'r') as f:
# with open('test.txt', 'r') as f:
    datatxt = [parse_row(row) for row in f]

# Create a DataFrame from the list of dictionaries
dftxt = pd.DataFrame(datatxt)
# dftxt


# Save the DataFrame to a CSV file
dftxt.to_csv(filename.replace('.csv', '') + '_imputed.csv', index=False)  # Save without index



In [ ]:
def regenerate_data(normalized_df, mins, maxs):
    """
    Regenerates the original data from the normalized data, minimum, and maximum values.

    Args:
        normalized_df: A pandas DataFrame with normalized values.
        mins: A pandas Series containing the minimum values of each column.
        maxs: A pandas Series containing the maximum values of each column.

    Returns:
        A pandas DataFrame with the original data.
    """

    # Regenerate original data
    original_df = (normalized_df * (maxs - mins)) + mins

    return original_df



mins = pd.read_csv('cal_dataframe_mins.csv')
maxs = pd.read_csv('cal_dataframe_maxs.csv')
result_0 = regenerate_data(dftxt, mins, maxs)
result_0